# Thông tin sinh viên:
***Họ và tên***: Trần Mỹ Hiền Trang

***Mssv***: 18021296

***Lop***: K63K1

## Giải nén dữ liệu

In [ ]:
!unzip ../input/home-depot-product-search-relevance/product_descriptions.csv.zip
!unzip ../input/home-depot-product-search-relevance/train.csv.zip
!unzip ../input/home-depot-product-search-relevance/test.csv.zip
!unzip ../input/home-depot-product-search-relevance/attributes.csv.zip

## Thêm thư viện

In [ ]:
from sklearn import pipeline
import numpy as np 
import pandas as pd 
from subprocess import check_output
import os 
import json
import warnings; warnings.filterwarnings("ignore");
import time
start_time = time.time()
from sklearn.ensemble import RandomForestRegressor
from sklearn import pipeline
from sklearn.model_selection import GridSearchCV 
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, make_scorer
from nltk.stem.porter import *
stemmer = PorterStemmer()
from bs4 import BeautifulSoup
import re
import random
random.seed(2016)
from scipy.stats import norm  
import seaborn as sns 
%matplotlib inline
import matplotlib.pyplot as plt

## Tải dữ liệu từ tệp CSV vào Pandas DataFrame

In [ ]:
df_train = pd.read_csv('train.csv', encoding="ISO-8859-1") #cập nhật tại đây
df_test = pd.read_csv('test.csv', encoding="ISO-8859-1") #cập nhật tại đây
df_pro_desc = pd.read_csv('product_descriptions.csv')[:64000] #cập nhật tại đây
df_attr = pd.read_csv('attributes.csv')

### Nhận thêm một số thông tin về dữ liệu:

Kiểm tra giá trị null trong mỗi khung dữ liệu

In [ ]:
print(f"training data has {df_train.isnull().values.sum()} null values:")
print(f"testing data has {df_test.isnull().values.sum()} null values:")
print(f"attribute data has {df_attr.isnull().values.sum()} null values:")
print(f"description data has {df_pro_desc.isnull().values.sum()} null values:")

### Phân tích dữ liệu đào tạo

In [ ]:
def get_df_info(df):
    print("df columns: \n",df.columns)
    print(f"df shape: \n",df.shape)
    print(f"df data types: \n",df.dtypes)
    return df.head(10)

In [ ]:
get_df_info(df_train)

In [ ]:
print("there are in total {} products ".format(len(df_train.product_title.unique())))
print("there are in total {} search query ".format(len(df_train.search_term.unique())))
print("there are in total {} product_uid".format(len(df_train.product_uid.unique())))

### Phân tích dữ liệu kiểm tra

In [ ]:
get_df_info(df_test)

In [ ]:
print("there are in total {} products ".format(len(df_test.product_title.unique())))
print("there are in total {} search query ".format(len(df_test.search_term.unique())))
print("there are in total {} product_uid".format(len(df_test.product_uid.unique())))

### Phân tích dữ liệu mô tả sản phẩm

Chứa `product_id` và` product_description`

`product_description`  chứa thông tin có giá trị và có thể hữu ích sau này

In [ ]:
get_df_info(df_pro_desc)

In [ ]:
print("there are in total {} product_uid ".format(len(df_pro_desc.product_uid.unique())))
print("there are in total {} product_descriptions ".format(len(df_pro_desc.product_description.unique())))

In [ ]:
(df_pro_desc.product_description.str.count('\d+') + 1).hist(bins=30)
(df_pro_desc.product_description.str.count('\W')+1).hist(bins=30)

### Tham gia đào tạo và kiểm tra với 'product_description' bằng 'product_uid'

In [ ]:
train_merged = pd.merge(df_train, df_pro_desc, on='product_uid')
test_merged = pd.merge(df_test, df_pro_desc, on='product_uid')

## **Tìm hiểu chi tiết**
**`relevance`** (Mức độ liên quan)

> `relevance` là mục tiêu của chúng ta

Mức độ liên quan là một số nằm trong khoảng từ 1 (không liên quan) đến 3 (mức độ liên quan cao)

In [ ]:
sns.countplot(train_merged.relevance)

In [ ]:
hight_relevance = train_merged.loc[train_merged["relevance"] == 3]
hight_relevance[['product_title', 'search_term']].head()

In [ ]:
train_merged.relevance.plot(kind='hist', density=True)

mu, std = norm.fit(train_merged.relevance)

xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)
title = "Results: mu = %.3f,  std = %.3f" % (mu, std)
plt.title(title)

plt.show()

Mức độ liên quan trong khoảng từ 1 đến 3. Vì mật độ sản phẩm có mức độ liên quan trong khoảng từ 2 đến 3 cao hơn, chúng ta có thể kết luận rằng hầu hết truy vấn tìm kiếm đã được phân loại từ 2 đến 3

Biểu đồ của mức độ liên quan không tuân theo mô hình phân phối chuẩn

**`search_term`**

`search_term` là những từ mà khách hàng sử dụng để tìm kiếm sản phẩm.

In [ ]:
train_merged[train_merged.search_term.str.contains("^\\d+ . \\d+$")].head(10)

In [ ]:
df_train_distribution = train_merged.search_term.str.split().apply(len).value_counts().sort_index()
df_test_distribution = test_merged.search_term.str.split().apply(len).value_counts().sort_index()
fig, (ax1, ax2) = plt.subplots(2, sharex=False)
fig.suptitle('Số lượng từ trong cụm từ tìm kiếm của tệp train và test')
df_train_distribution.plot(kind = 'line',colormap = 'jet',sharex = True, figsize = (10,5), ax = ax1, title = 'dữ liệu train')
df_test_distribution.plot(kind = 'line', colormap = 'spring', figsize = (10,5), ax = ax2, title = 'dữ liệu test')


Một số truy vấn trong đào tạo tập dữ liệu quá đơn giản, thật khó để đoán chính xác ý của người dùng theo nghĩa rộng

Một số truy vấn tìm kiếm trong đào tạo tập dữ liệu lại quá cụ thể 

Số lượng lần xuất hiện trong tiêu đề sản phẩm có xu hướng nhiều hơn số lượng ký tự để đào tạo tập dữ liệu (và điều này cũng đúng với trường truy vấn tìm kiếm)

`product_description`

In [ ]:
df_train_pro_desc_distribution = train_merged.product_description.str.split().apply(len).divmod(10)[0].value_counts().nlargest(40).sort_index()
df_test_pro_desc_distribution = test_merged.product_description.str.split().apply(len).divmod(10)[0].value_counts().nlargest(40).sort_index()
fig, (ax1, ax2) = plt.subplots(2)
fig.suptitle('Số lượng từ trong mô tả sản phẩm của tập train và tập test')

df_train_pro_desc_distribution.plot(kind = 'line', colormap = 'jet',sharex = True, figsize = (10,5), ax = ax1, title = 'tại tệp train')
df_test_pro_desc_distribution.plot(kind = 'line', colormap = 'spring', figsize = (10, 5), ax = ax2, title = 'tại tệp test')

# Xử lý dữ liệu

## Chuyển đổi `dt_attributes`
- Chúng ta sẽ chuyển đổi các thuộc tính thành khung dữ liệu mới chứa hai cột `product_uid` và` brand`

- Vì không phải mọi trường đều có `thuộc tính sản phẩm`, khi chúng ta hợp nhất chúng với` dt_description`, một số giá trị `NAN` sẽ hiển thị. Do đó, chúng ta cần xử lý các giá trị `NAN` sau đó.


### Hợp nhất tất cả

Để có cái nhìn tổng quát về dữ liệu, chúng ta sẽ hợp nhất tập train và test.

In [ ]:
df_brand = df_attr[df_attr.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand"})
num_train = df_train.shape[0]
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')
df_all = pd.merge(df_all, df_brand, how='left', on='product_uid')

In [ ]:
df_all.columns

In [ ]:
from nltk.corpus import stopwords # Import the stop word list

### Làm sạch dữ liệu

Đầu tiên, xóa các thẻ html

In [ ]:
# sử dụng Beautifulsoup lib để xóa các thẻ html trong văn bản
def remove_html_tag(text):
    soup = BeautifulSoup(text, 'lxml')
    text = soup.get_text().replace('Click here to review our return policy for additional information regarding returns', '')
    return text

Sau đó, chúng ta loại bỏ các ký tự đặc biệt được cho là chứa rất ít thông tin và tiêu chuẩn hóa các đơn vị đo 

In [ ]:
#stopwords là những từ chứa rất ít hoặc không có thông tin.
stop_w = ['for', 'xbi', 'and', 'in', 'th','on','sku','with','what','from','that','less','er','ing'] #'electr','paint','pipe','light','kitchen','wood','outdoor','door','bathroom'

strNum = {'zero':0,'one':1,'two':2,'three':3,'four':4,'five':5,'six':6,'seven':7,'eight':8,'nine':9}
spell_check = json.load(open('../input/spell-check/spell_check.json','r'))
def str_stem(s):
    if isinstance(s, str):
        s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) #Split words with a.A
        s = s.lower()
        s = s.replace("  "," ") #xóa khoảng trắng bị nhân đôi
        # xóa ký tự đặc biệt và tách số
        s = s.replace(",","") #could be number / segment later
        s = s.replace("$"," ")
        s = s.replace("?"," ")
        s = s.replace("-"," ")
        s = s.replace("//","/")
        s = s.replace("..",".")
        s = s.replace(" / "," ")
        s = s.replace(" \\ "," ")
        s = s.replace("."," . ")
        s = re.sub(r"(^\.|/)", r"", s)
        s = re.sub(r"(\.|/)$", r"", s)
        s = re.sub(r"([0-9])([a-z])", r"\1 \2", s)
        s = re.sub(r"([a-z])([0-9])", r"\1 \2", s)
        s = s.replace(" x "," xbi ")
        s = re.sub(r"([a-z])( *)\.( *)([a-z])", r"\1 \4", s)
        s = re.sub(r"([a-z])( *)/( *)([a-z])", r"\1 \4", s)
        s = s.replace("*"," xbi ")
        s = s.replace(" by "," xbi ")
        #chuyển đổi đơn vị đo lường sang dạng chuẩn.
        s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
        s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
        s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
        s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
        s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
        s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
        s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
        s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
        s = s.replace("°"," degrees ")
        s = re.sub(r"([0-9]+)( *)(degrees|degree)\.?", r"\1deg. ", s)
        s = s.replace(" v "," volts ")
        s = re.sub(r"([0-9]+)( *)(volts|volt)\.?", r"\1volt. ", s)
        s = re.sub(r"([0-9]+)( *)(watts|watt)\.?", r"\1watt. ", s)
        s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp. ", s)
        s = s.replace("  "," ")
        s = s.replace(" . "," ")
        # lọc ra các từ dừng
        s = (" ").join([z for z in s.split(" ") if z not in stop_w])
        # chuyển đổi chuỗi số thành số
        s = (" ").join([str(strNum[z]) if z in strNum else z for z in s.split(" ")])
        s = (" ").join([stemmer.stem(z) for z in s.split(" ")])
        s = remove_html_tag(s)
        s = s.lower()
        # sửa lỗi chính tả bị thiếu trong văn bản
        for (k,v) in spell_check.items():
            s = s.replace(k,v)
        return s
    else:
        return "null"

Áp dụng hàm tiền xử lý cho `product_title`,` search_term`, `product_description` và` brand`

In [ ]:
df_all['search_term'] = df_all['search_term'].map(lambda x:str_stem(x))
df_all['product_title'] = df_all['product_title'].map(lambda x:str_stem(x))
df_all['product_description'] = df_all['product_description'].map(lambda x:str_stem(x))
df_all['brand'] = df_all['brand'].map(lambda x:str_stem(x))

### Tạo trường mới
Chúng ta có thể làm phong phú thêm thông tin bằng cách xây dựng các tính năng mới từ dữ liệu thô
Ví dụ: chúng ta sẽ có `product_info` nếu nối` search_term`, `product_title` với` product_description` được phân tách bằng `tab`

In [ ]:
df_all['product_info'] = df_all['search_term']+"\t"+df_all['product_title'] +"\t"+df_all['product_description']

Tạo số cột dựa trên cột văn bản

* đếm số từ trong `search_term`
* đếm số từ trong `product_title`
* đếm số từ trong `product description`
* đếm số từ trong `brand`

In [ ]:
df_all['len_of_query'] = df_all['search_term'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_title'] = df_all['product_title'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_description'] = df_all['product_description'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_brand'] = df_all['brand'].map(lambda x:len(x.split())).astype(np.int64)

In [ ]:
def seg_words(str1, str2):
    '''
    str1: search_term
    str2: product_tilte
    '''
    str2 = str2.lower()
    str2 = re.sub("[^a-z0-9./]"," ", str2)
    str2 = [s for s in set(str2.split()) if len(s)>2]
    words = str1.lower().split(" ")
    s = []
    for word in words:
        if len(word)>3:
            s1 = []
            s1 += segmentit(word,str2,True)
            if len(s)>1:
                s += [z for z in s1 if z not in ['er','ing','s','less'] and len(z)>1]
            else:
                s.append(word)
        else:
            s.append(word)
    return (" ".join(s))

In [ ]:
def segmentit(s, txt_arr, t):
    st = s
    r = []
    for j in range(len(s)):
        for word in txt_arr:
            if word == s[:-j]:
                r.append(s[:-j])
                s=s[len(s)-j:]
                r += segmentit(s, txt_arr, False)
    if t:
        i = len(("").join(r))
        if not i==len(st):
            r.append(st[i:])
    return r

In [ ]:
df_all['search_term'] = df_all['product_info'].map(lambda x:seg_words(x.split('\t')[0],x.split('\t')[1]))

In [ ]:
def str_common_word(str1, str2):
    words, cnt = str1.split(), 0
    for word in words:
        if str2.find(word)>=0:
            cnt+=1
    return cnt

## Numeric feature: 
### Đếm

In [ ]:
#đếm số lần từ cuối cùng trong tiêu đề xuất hiện trong cụm từ tìm kiếm
df_all['query_last_word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[1]))
#đếm số lần từ cuối cùng trong product_description xuất hiện trong cụm từ tìm kiếm
df_all['query_last_word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[2]))
#đếm số lần mỗi từ trong product_info xuất hiện trong cụm từ tìm kiếm
df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
#đếm số lần mỗi từ trong product_description xuất hiện trong cụm từ tìm kiếm
df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))

In [ ]:
#đếm số lần mỗi từ trong string1 xuất hiện trong string2
def str_whole_word(str1, str2, i_):
    cnt = 0
    while i_ < len(str2):
        i_ = str2.find(str1, i_)
        if i_ == -1:
            return cnt
        else:
            cnt += 1
            i_ += len(str1)
    return cnt

In [ ]:
df_all['query_in_title'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[1],0))
df_all['query_in_description'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[2],0))

### Tính toán tỷ lệ

In [ ]:
df_all['ratio_title'] = df_all['word_in_title']/df_all['len_of_query']
df_all['ratio_description'] = df_all['word_in_description']/df_all['len_of_query']
df_all['attr'] = df_all['search_term']+"\t"+df_all['brand']
df_all['word_in_brand'] = df_all['attr'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['ratio_brand'] = df_all['word_in_brand']/df_all['len_of_brand']

# String similarity feature

In [ ]:
!pip install textdistance

In [ ]:
import textdistance
df_all['jaccard_sim_desc'] = df_all['product_info'].map(lambda x:textdistance.jaccard(x.split('\t')[0],x.split('\t')[2]))
df_all['jaccard_sim_title'] = df_all['product_info'].map(lambda x:textdistance.jaccard(x.split('\t')[0],x.split('\t')[1]))

# Khoảng cách Levenshtein

In [ ]:
df_all['levenshtein_sim_desc'] = df_all['product_info'].map(lambda x:textdistance.levenshtein(x.split('\t')[0],x.split('\t')[2]))
df_all['levenshtein_sim_title'] = df_all['product_info'].map(lambda x:textdistance.levenshtein(x.split('\t')[0],x.split('\t')[1]))

In [ ]:
df_all['mra_sim_desc'] = df_all['product_info'].map(lambda x:textdistance.mra(x.split('\t')[0],x.split('\t')[2]))
df_all['mra_sim_title'] = df_all['product_info'].map(lambda x:textdistance.mra(x.split('\t')[0],x.split('\t')[1]))

In [ ]:
df_brand = pd.unique(df_all.brand.ravel())
d={}
i = 1000
for s in df_brand:
    d[s]=i
    i+=3
df_all['brand_feature'] = df_all['brand'].map(lambda x:d[x])
df_all['search_term_feature'] = df_all['search_term'].map(lambda x:len(x))


# Word embedding feature

In [ ]:
from nltk.corpus import brown
import gensim
embed_model = gensim.models.Word2Vec(brown.sents())
embed_model.save('brown.embedding')
model = gensim.models.Word2Vec.load('brown.embedding')

In [ ]:
def embedding_sim_cal(s, t, i):
    _sum = 0
    avg = 0
    if len(s.split()) == 0 :
        return 0
    for s_word in s.split():
        _max = 0
        for t_word in t.split():
            if ((s_word in model.wv) and (t_word in model.wv)):
                _max = max(_max, model.wv.similarity(s_word, t_word))
        _sum += _max
    avg = _sum/ len(s.split())
    return avg

In [ ]:
df_all['word_ebed_similarity'] = df_all['product_info'].map(lambda x:embedding_sim_cal(x.split('\t')[0],x.split('\t')[2],0))
df_all.to_csv('df_all.csv')

# Tính năng đo độ tương tự TF-IDF ở cấp ký tự

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tfidf = TfidfVectorizer(analyzer='char_wb', ngram_range = (3,3), max_features = 1500)
tfidf_des = tfidf.fit_transform(df_all.product_description).toarray()
tfidf_search = tfidf.transform(df_all.search_term).toarray()
tfidf = TfidfVectorizer(ngram_range=(1, 1), stop_words='english')

# Mô Hình Hóa
## Hàm Loss: RMSE

In [ ]:
def fmean_squared_error(ground_truth, predictions):
    fmean_squared_error_ = mean_squared_error(ground_truth, predictions)**0.5
    return fmean_squared_error_

RMSE  = make_scorer(fmean_squared_error, greater_is_better=False)

In [ ]:
class custom_regression_vals(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    def transform(self, hd_searches):
        d_col_drops=['id','relevance','search_term','product_title','product_description','product_info','attr','brand']
        hd_searches = hd_searches.drop(d_col_drops,axis=1).values
        return hd_searches

class custom_txt_col(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, x, y=None):
        return self
    def transform(self, data_dict):
        return data_dict[self.key].apply(str)

Chia dữ liệu thành dữ liệu train và test

In [ ]:
df_train = df_all.iloc[:num_train]
df_test = df_all.iloc[num_train:]
id_test = df_test['id']
y_train = df_train['relevance'].values
X_train = df_train[:]
X_test = df_test[:]
print("--- Features Set: %s minutes ---" % round(((time.time() - start_time)/60),2))

# Random Forest Regressor

In [ ]:
rfr = RandomForestRegressor(n_estimators = 530, n_jobs = -1, random_state = 2016, verbose = 1)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor()

### Giảm kích thước bằng cắt ngắn SVD (hay còn gọi là LSA).


In [ ]:
tsvd = TruncatedSVD(n_components=10, random_state = 2016)

## Tạo đường dẫn

In [ ]:
clf = pipeline.Pipeline([
        ('union', FeatureUnion(
                    transformer_list = [
                        ('cst',  custom_regression_vals()),  
                        ('txt1', pipeline.Pipeline([('s1', custom_txt_col(key='search_term')), ('tfidf1', tfidf), ('tsvd1', tsvd)])),
                        ('txt2', pipeline.Pipeline([('s2', custom_txt_col(key='product_title')), ('tfidf2', tfidf), ('tsvd2', tsvd)])),
                        ('txt3', pipeline.Pipeline([('s3', custom_txt_col(key='product_description')), ('tfidf3', tfidf), ('tsvd3', tsvd)])),
                        ('txt4', pipeline.Pipeline([('s4', custom_txt_col(key='brand')), ('tfidf4', tfidf), ('tsvd4', tsvd)]))
                        ],
                    transformer_weights = {
                        'cst': 1.0,
                        'txt1': 0.5,
                        'txt2': 0.25,
                        'txt3': 0.01,
                        'txt4': 0.5
                        },
                )), 
        ('rfr', rfr)])
param_grid = {'rfr__max_features': [8], 'rfr__max_depth': [18]}

## Sử dụng GridSearchCV để lựa chọn mô hình

In [ ]:
model = GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs = -1, cv = 2, verbose = 20, scoring=RMSE)
model.fit(X_train, y_train)

# Submission

In [ ]:
pd.DataFrame({"id": id_test, "relevance": model.predict(X_test)}).to_csv('submission.csv',index=False)